Importing Python Libraries and preparing the environment

In [85]:
!pip install transformers seqeval[gpu]

In [86]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification

The run time parameters here are changed to GPU (for acceleration)


In [87]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


Dataset preprocessing

In [88]:
data = pd.read_csv("ner_datasetreference.csv", encoding='unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [89]:
data.count()

,0
Sentence #,47959
Word,1048565
POS,1048575
Tag,1048575


In [90]:
data = data[:30000]

In [91]:
data.count()

,0
Sentence #,1344
Word,30000
POS,30000
Tag,30000


In [92]:
print("Number of tags: {}".format(len(data.Tag.unique())))
frequencies = data.Tag.value_counts()
frequencies

Number of tags: 17


,count
Tag,
O,25586
B-geo,821
B-gpe,650
B-org,602
I-per,524
B-tim,516
B-per,450
I-org,407
I-geo,155


For my task I need to identify only Locations and Dates, so the rest of tags should be ignored. The big plus of this dataset - it already have LOCATION and DATE as Entities (a lot of other reviewed datasets does not have an Date param)

In [93]:
data[data["Tag"] == "I-gpe"].head(100)

,Sentence #,Word,POS,Tag
1225,NaN,States,NNPS,I-gpe
1264,NaN,Korea,NNP,I-gpe
2713,NaN,Binh,NNP,I-gpe
2932,NaN,Ababa,NNP,I-gpe
3466,NaN,City,NNP,I-gpe
5241,NaN,Lanka,NNP,I-gpe
5313,NaN,Korea,NNP,I-gpe
5361,NaN,Korea,NNP,I-gpe
5370,NaN,Korea,NNP,I-gpe
5390,NaN,Korea,NNP,I-gpe


I need I-tim and B-tim as DATE, I-geo and B-geo, I-gpe and B-gpe as LOCATION, for clarity i would set rest of tags to "O" and change geo and gpe to loc as well for simplicity.

In [94]:
data["Tag"] = data["Tag"].replace({
    "B-geo": "B-LOC",
    "I-geo": "I-LOC",
    "I-gpe": "I-LOC",
    "B-gpe": "B-LOC"
})

In [95]:
data[data["Tag"] == "I-LOC"].head(100)

,Sentence #,Word,POS,Tag
66,NaN,Park,NNP,I-LOC
347,NaN,State,NNP,I-LOC
350,NaN,State,NNP,I-LOC
381,NaN,Delta,NNP,I-LOC
561,NaN,Arab,NNP,I-LOC
...,...,...,...,...
17252,NaN,States,NNPS,I-LOC
17282,NaN,U.S.,NNP,I-LOC
17406,NaN,Reconstruction,NNP,I-LOC
17844,NaN,Ocean,NNP,I-LOC


In [96]:
data["Tag"] = data["Tag"].replace({
    "B-tim": "B-DATE",
    "I-tim": "I-DATE"
})

In [97]:
data[data["Tag"] == "B-DATE"].head(100)

,Sentence #,Word,POS,Tag
167,NaN,Wednesday,NNP,B-DATE
211,NaN,Wednesday,NNP,B-DATE
274,NaN,Tuesday,NNP,B-DATE
341,NaN,Wednesday,NNP,B-DATE
493,NaN,Wednesday,NNP,B-DATE
...,...,...,...,...
7086,NaN,Monday,NNP,B-DATE
7135,NaN,Monday,NNP,B-DATE
7301,NaN,July,NNP,B-DATE
7306,NaN,July,NNP,B-DATE


In [98]:
data["Tag"] = data["Tag"].replace({
    "B-org": "O",
    "I-org": "O",
    "B-per": "O",
    "I-per": "O",
    "B-art": "O",
    "I-art": "O",
    "B-eve": "O",
    "I-eve": "O",
    "B-nat": "O",
    "I-nat": "O"
})

In [99]:
print("Number of tags: {}".format(len(data.Tag.unique())))
frequencies = data.Tag.value_counts()
frequencies

Number of tags: 5


,count
Tag,
O,27699
B-LOC,1471
B-DATE,516
I-LOC,182
I-DATE,132


In [100]:
data = data.fillna(method='ffill')
data['sentence'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))
data['word_labels'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Tag'].transform(lambda x: ','.join(x))
data.head(100)

/tmp/ipython-input-3284187623.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')


,Sentence #,Word,POS,Tag,sentence,word_labels
0,Sentence: 1,Thousands,NNS,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-LOC,O,O,O,O,O,B-LOC,O,O,O,O,O,B-..."
1,Sentence: 1,of,IN,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-LOC,O,O,O,O,O,B-LOC,O,O,O,O,O,B-..."
2,Sentence: 1,demonstrators,NNS,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-LOC,O,O,O,O,O,B-LOC,O,O,O,O,O,B-..."
3,Sentence: 1,have,VBP,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-LOC,O,O,O,O,O,B-LOC,O,O,O,O,O,B-..."
4,Sentence: 1,marched,VBN,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-LOC,O,O,O,O,O,B-LOC,O,O,O,O,O,B-..."
...,...,...,...,...,...,...
95,Sentence: 5,'s,POS,O,The protest comes on the eve of the annual con...,"O,O,O,O,O,O,O,O,O,O,O,B-LOC,O,O,O,O,O,O,O,B-LO..."
96,Sentence: 5,ruling,VBG,O,The protest comes on the eve of the annual con...,"O,O,O,O,O,O,O,O,O,O,O,B-LOC,O,O,O,O,O,O,O,B-LO..."
97,Sentence: 5,Labor,NNP,O,The protest comes on the eve of the annual con...,"O,O,O,O,O,O,O,O,O,O,O,B-LOC,O,O,O,O,O,O,O,B-LO..."
98,Sentence: 5,Party,NNP,O,The protest comes on the eve of the annual con...,"O,O,O,O,O,O,O,O,O,O,O,B-LOC,O,O,O,O,O,O,O,B-LO..."


In [101]:
label2id = {k: v for v, k in enumerate(data.Tag.unique())}
id2label = {v: k for v, k in enumerate(data.Tag.unique())}
label2id

{'O': 0, 'B-LOC': 1, 'I-LOC': 2, 'B-DATE': 3, 'I-DATE': 4}

In [102]:
data.iloc[5].sentence

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [103]:
data.iloc[5].word_labels

'O,O,O,O,O,O,B-LOC,O,O,O,O,O,B-LOC,O,O,O,O,O,B-LOC,O,O,O,O,O'

Preparing the dataset and dataloader

In [104]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [105]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")

In [106]:
def label_tokenize(sentence, text_labels, tokenizer):
    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)
        tokenized_sentence.extend(tokenized_word)
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [107]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]
        word_labels = self.data.word_labels[index]
        tokenized_sentence, labels = label_tokenize(sentence, word_labels, self.tokenizer)

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [108]:
train_size = 0.7
val_size = 0.2
test_size = 0.1

train_val_dataset = data.sample(frac=(train_size + val_size), random_state=200)
test_dataset = data.drop(train_val_dataset.index).reset_index(drop=True)

train_dataset = train_val_dataset.sample(frac=train_size / (train_size + val_size), random_state=200)
val_dataset = train_val_dataset.drop(train_dataset.index).reset_index(drop=True)

train_dataset = train_dataset.reset_index(drop=True)
val_dataset = val_dataset.reset_index(drop=True)

print("FULL Dataset:", data.shape)
print("TRAIN Dataset:", train_dataset.shape)
print("VALIDATION Dataset:", val_dataset.shape)
print("TEST Dataset:", test_dataset.shape)

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
validation_set = dataset(val_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (30000, 6)
TRAIN Dataset: (21000, 6)
VALIDATION Dataset: (6000, 6)
TEST Dataset: (3000, 6)


In [109]:
training_set[44]

{'ids': tensor([ 101, 2002, 2056, 2107, 6256, 3216, 4675, 2000, 1996, 2204, 2458, 1997,
         1057, 1012, 1055, 1011, 2859, 4262, 1012,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0,

In [110]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:30]), training_set[0]["targets"][:30]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

[CLS]       O
norway      B-LOC
'           O
s           O
prime       O
minister    O
jens        O
st          O
##olt       O
##enberg    O
and         O
nobel       O
-           O
prize       O
winning     O
environmental  O
##ist       O
wang        O
##ari       O
ma          O
##ath       O
##ai        O
on          O
tuesday     B-DATE
placed      O
the         O
first       O
seeds       O
inside      O
the         O


In [111]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

val_params = {
    'batch_size': VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
}

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
val_loader = DataLoader(validation_set, **val_params)

Model Set-Up

In [112]:
model = BertForTokenClassification.from_pretrained(
    'prajjwal1/bert-tiny',
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, 

Karpathy test

In [113]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(1.6766, device='cuda:0', grad_fn=<NllLossBackward0>)

The Karpathy test is passed, we have 5 classes, −ln(1/5)≈1.609 is close to my result - 1,68

In [114]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 5])

In [115]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [116]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []

    # Put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype=torch.long)
        mask = batch['mask'].to(device, dtype=torch.long)
        targets = batch['targets'].to(device, dtype=torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100 == 0:
            loss_step = tr_loss / nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}, num {idx}")

        # Compute training accuracy
        flattened_targets = targets.view(-1)  # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels)  # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1)  # shape (batch_size * seq_len,)

        active_accuracy = mask.view(-1) == 1  # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        targets = targets.to(device)
        predictions = predictions.to(device)


        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")


In [117]:
def validate(model, validation_loader):
    # Put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(validation_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100 == 0:
                loss_step = eval_loss / nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # Compute evaluation accuracy
            flattened_targets = targets.view(-1)  # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels)  # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1)  # shape (batch_size * seq_len,)

            # Mask out padding tokens
            active_accuracy = mask.view(-1) == 1  # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return eval_loss, eval_accuracy, eval_labels, eval_preds


In [118]:
def test(model, testing_loader):
    # Put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100 == 0:
                loss_step = eval_loss / nb_eval_steps
                print(f"Test loss per 100 evaluation steps: {loss_step}")

            # Compute evaluation accuracy
            flattened_targets = targets.view(-1)  # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels)  # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1)  # shape (batch_size * seq_len,)

            # Mask out padding tokens
            active_accuracy = mask.view(-1) == 1  # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Test Loss: {eval_loss}")
    print(f"Test Accuracy: {eval_accuracy}")

    return eval_loss, eval_accuracy, eval_labels, eval_preds


In [120]:
for epoch in range(EPOCHS):
    # Train the model
    train(epoch)

    # Validate the model after each epoch
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    val_loss, val_accuracy, val_labels, val_preds = validate(model, val_loader)

    # Optionally, you can save the model after each epoch
    # torch.save(model.state_dict(), 'model_epoch_{}.pth'.format(epoch + 1))

    # Print or track validation loss and accuracy
    print(f"Validation Loss: {val_loss}")
    print(f"Validation Accuracy: {val_accuracy}")

# After training, you can test on the test set
test_loss, test_accuracy, test_labels, test_preds = test(model, testing_loader)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Training loss per 100 training steps: 0.0664013922214508, num 0
Training loss per 100 training steps: 0.0617680598677385, num 100
Training loss per 100 training steps: 0.05927988189962966, num 200
Training loss per 100 training steps: 0.05781143735413535, num 300
Training loss per 100 training steps: 0.05754334171142364, num 400
Training loss per 100 training steps: 0.05670150467928893, num 500
Training loss per 100 training steps: 0.0557431617221747, num 600
Training loss per 100 training steps: 0.05507921691872339, num 700
Training loss per 100 training steps: 0.054216356336437954, num 800
Training loss per 100 training steps: 0.053450098600275775, num 900
Training loss per 100 training steps: 0.05268083281487554, num 1000
Training loss per 100 training steps: 0.05208890863629117, num 1100
Training loss per 100 training steps: 0.051202690999623046, num 1200
Training loss per 100 training steps: 0.050445795893886744, num 1300
Training loss epoch: 0.05042203691534795
Training accuracy 

In [121]:
torch.save(model.state_dict(), 'model.pth')

Test Cases

In [122]:
def predict_ner_line(model, tokenizer, text, id2label, max_len=128, device='cuda'):
    model.eval(); model.to(device)
    enc = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_len).to(device)

    with torch.no_grad():
        logits = model(**enc).logits
        pred_ids = logits.argmax(dim=2)[0]

    input_ids = enc["input_ids"][0].tolist()
    tokens = tokenizer.convert_ids_to_tokens(input_ids, skip_special_tokens=False)

    out = []
    for tok, pid in zip(tokens, pred_ids.tolist()):
        if tok in ("[CLS]", "[SEP]", "[PAD]"):
            continue
        out.append((tok, id2label[pid]))
    return out




In [123]:
test_case = "I was born in Lutsk, Ukraine. On November 4, 2005"

In [128]:
predict_ner_line(model, tokenizer, test_case, id2label, max_len=128, device=device)

[('i', 'O'),
 ('was', 'O'),
 ('born', 'O'),
 ('in', 'O'),
 ('lu', 'B-LOC'),
 ('##tsk', 'B-LOC'),
 (',', 'B-DATE'),
 ('ukraine', 'B-LOC'),
 ('.', 'O'),
 ('on', 'B-DATE'),
 ('november', 'B-DATE'),
 ('4', 'B-DATE'),
 (',', 'B-DATE'),
 ('2005', 'B-DATE')]

In [125]:
from google.colab import files
files.download('model.pth')  # Replace with your file path


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [126]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
import torch

def evaluate_from_loader(model, id2label, test_loader, device='cuda'):
    model.eval(); model.to(device)

    y_true, y_pred = [], []

    with torch.no_grad():
        for batch in test_loader:
            ids     = batch['ids'].to(device)
            mask    = batch['mask'].to(device)
            targets = batch['targets'].to(device)   # (B, L) з -100 для спец. токенів

            outputs = model(input_ids=ids, attention_mask=mask)
            pred_ids = outputs.logits.argmax(-1)    # (B, L)

            # перетворюємо у списки BIO, ігноруючи -100
            for true_row, pred_row in zip(targets, pred_ids):
                true_labels_seq, pred_labels_seq = [], []
                for t_id, p_id in zip(true_row.tolist(), pred_row.tolist()):
                    if t_id == -100:
                        continue
                    true_labels_seq.append(id2label[t_id])
                    pred_labels_seq.append(id2label[p_id])
                y_true.append(true_labels_seq)
                y_pred.append(pred_labels_seq)

    p = precision_score(y_true, y_pred)
    r = recall_score(y_true, y_pred)
    f = f1_score(y_true, y_pred)
    print(classification_report(y_true, y_pred))
    return p, r, f


In [127]:
p, r, f = evaluate_from_loader(model, id2label, testing_loader)
print(f"Precision: {p:.4f} | Recall: {r:.4f} | F1: {f:.4f}")


              precision    recall  f1-score   support

        DATE       0.65      0.74      0.69      1385
         LOC       0.80      0.91      0.85      5025

   micro avg       0.77      0.88      0.82      6410
   macro avg       0.72      0.83      0.77      6410
weighted avg       0.77      0.88      0.82      6410

Precision: 0.7667 | Recall: 0.8780 | F1: 0.8186
